In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [4]:
# df = pd.read_csv('../../data/annotation_100_input_241201.csv')

In [5]:
df_one = pd.read_csv('../../data/part_one_annotation_50_input.csv')
df_two = pd.read_csv('../../data/part_two_annotation_50_input.csv')

In [6]:
df_one.head(10)

,id,link,Q_difficulty,A_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate,Intermediate
5,76992629,https://stackoverflow.com/questions/76992629,Intermediate,Advanced
6,77370805,https://stackoverflow.com/questions/77370805,Intermediate,Intermediate
7,76784524,https://stackoverflow.com/questions/76784524,Basic,Basic
8,76608783,https://stackoverflow.com/questions/76608783,Intermediate,Basic
9,76697269,https://stackoverflow.com/questions/76697269,Intermediate,Intermediate


In [7]:
df_one = df_one[~(df_one['Q_difficulty']=='-') & ~(df_one['A_difficulty']=='-') ].reset_index(drop=True)
df_two = df_two[~(df_two['Q_difficulty']=='-') & ~(df_two['A_difficulty']=='-') ].reset_index(drop=True)

In [8]:
df = pd.concat([df_one, df_two ], axis = 0)

In [9]:
df.head()

,id,link,Q_difficulty,A_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate,Intermediate


In [10]:
# 'id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set',
#        'Q_Granular_breakdown'

In [11]:
q_df = df[['id', 'link', 'Q_difficulty']]
a_df = df[['id', 'link', 'A_difficulty']]

In [12]:
q_df.head()

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate


In [13]:
q_df

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate
...,...,...,...
28,74793352,https://stackoverflow.com/questions/74793352,Basic
29,74727945,https://stackoverflow.com/questions/74727945,Intermediate
30,74700764,https://stackoverflow.com/questions/74700764,Basic
31,77538219,https://stackoverflow.com/questions/77538219,Basic


In [14]:
q_id_str = str([x for x in q_df['id']])[1:-1]
a_id_str = str([x for x in a_df['id']])[1:-1]

In [15]:
q_id_str

'77293473, 77404924, 77249095, 77076663, 77019129, 76992629, 77370805, 76784524, 76608783, 76697269, 76317584, 76570219, 76302571, 76548866, 76541602, 76264395, 76169010, 76169378, 76059076, 76045819, 76085427, 75779691, 75862649, 75637544, 75460677, 75454689, 75283999, 75279786, 75198212, 75057003, 75179892, 75239843, 74778102, 74774389, 74760874, 74751254, 74693871, 74844950, 74804354, 77553477, 77543317, 77511053, 77467788, 77155599, 77291783, 77150535, 77068633, 77020489, 76917593, 76798081, 76774917, 76675433, 76427264, 76331128, 76367594, 76481433, 76115814, 76040179, 75870881, 75995957, 75711831, 75789316, 75942768, 75492157, 75352739, 75333570, 75266730, 75243277, 75022065, 74936129, 75099305, 74793352, 74727945, 74700764, 77538219, 77464645'

In [16]:
q_postypeid="'1'"
a_postypeid="'2'"



In [17]:
# q_sql = """select a.id 
#                 , 'Here is the question about software development. The title of the question is "'||a.title||'". and body of the question is "'||b.body||'". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'
#             from posts a 
#                , postsbody b
#             where a.id = b.id
#               and a.creationdate between '2021-11-30' and '2023-11-30' 
#               and a.posttypeid = {0} 
#               and a.tags like '%<python>%' 
#               and a.id in ({1})
# """

In [18]:
q_sql = """select a.id 
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [19]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [20]:
q_output = pd.DataFrame(rows, columns = [
  'id'
  ,'title'
  , 'body'
])


In [21]:
q_output.head()

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...
3,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
4,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...


In [22]:
q_output

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...
3,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
4,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...
...,...,...,...
71,77467788,Keep second regex lines in first regex lines m...,<p>I have high number of txt list files in <co...
72,77511053,How to install Resource package with Anaconda ...,<p>To install <code>Resource</code> package us...
73,77538219,How to write comprehension list in python,<p>I want to transform this :</p>\n<pre><code>...
74,77543317,wandb error when running as a subprocess: Mana...,<p>I’m having a problem when I try to run a su...


In [23]:
q_output['c_body'] = q_output['body'].apply(lambda x : codep(x))

In [24]:
q_output['t_body'] = q_output[['c_body', 'body']].apply(lambda row: ts.get_text_section({'body': row['body'], 'code_sections': row['c_body']['code_sections']}), axis=1)

In [25]:
q_output['t_body']

0     <p>According to <a href="https://keras.io/gett...
1     <p>I want to replace multiple strings in a lis...
2     <p>I have this function that I want to test:</...
3     <p>I am trying to iterate through a folder and...
4     <p>I need help for my school project. I am tak...
                            ...                        
71    <p>I have high number of txt list files in <co...
72    <p>To install <code>Resource</code> package us...
73    <p>I want to transform this :</p>\n<pre><code>...
74    <p>I’m having a problem when I try to run a su...
75    <p>I am a beginner to ML and am using Google C...
Name: t_body, Length: 76, dtype: object

In [26]:
q_output['clean_body'] = q_output['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))

In [27]:
q_output.head()

,id,title,body,c_body,t_body,clean_body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett...",{'code_sections': []},"<p>According to <a href=""https://keras.io/gett...","According to this keras documentation, pickle ..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...,"{'code_sections': [{'off_beg': 386, 'off_end':...",<p>I want to replace multiple strings in a lis...,I want to replace multiple strings in a list w...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...,"{'code_sections': [{'off_beg': 60, 'off_end': ...",<p>I have this function that I want to test:</...,I have this function that I want to test:\n\n\...
3,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...,"{'code_sections': [{'off_beg': 316, 'off_end':...",<p>I am trying to iterate through a folder and...,I am trying to iterate through a folder and de...
4,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...,"{'code_sections': [{'off_beg': 666, 'off_end':...",<p>I need help for my school project. I am tak...,I need help for my school project. I am taking...


In [28]:
q_output = pd.merge(q_output, q_df[['id', 'Q_difficulty']], on = ['id'] )

In [29]:
q_output['answer'] = 'Difficulty Level : '+q_output['Q_difficulty'].map(str)

In [30]:
q_output['question'] = """<Title>"""+q_output['title'].map(str)+"""</Title>. <Question>"""+q_output['clean_body'].map(str)+"""</Question>  Let's think through the difficulty of question carefully, step by step."""

In [31]:
import re
q_output['question'] = q_output['question'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
# .str.replace('";', "")

In [32]:
# q_output[['id', 'question', 'answer']].to_csv('../../data/q_output_241201.csv')

In [33]:
q_output[['id', 'question', 'answer']].to_csv('../../data/q_output.csv')

In [34]:
# [34,  3, 37, 33]
q_output[['body', 'clean_body']].to_csv('chk.csv')

In [35]:
# q_output.iloc[[14, 25, 13, 21], :].to_csv('chk.csv')